In [ ]:
# grp_opts.py
# group options based on right, symbol and strike.
# this makes it easy to delete unwanted ones on inspection.
import pandas as pd

def grp_opts(df):
    '''Groups options and sorts strikes by puts and calls
    Arg: 
       df as dataframe. Requires 'symbol', 'strike' and 'dte' fields in the df
    Returns: sorted dataframe'''
    
    gb = df.groupby('right')

    if 'C' in [k for k in gb.indices]:
        df_calls = gb.get_group('C').reset_index(drop=True).sort_values(['symbol', 'dte', 'strike'], ascending=[True, False, True])
    else:
        df_calls =  pd.DataFrame([])

    if 'P' in [k for k in gb.indices]:
        df_puts = gb.get_group('P').reset_index(drop=True).sort_values(['symbol', 'dte', 'strike'], ascending=[True, False, False])
    else:
        df_puts =  pd.DataFrame([])

    df = pd.concat([df_puts, df_calls]).reset_index(drop=True)
    
    return df

In [ ]:
# get_prec.py
# get precision, based on the base
from math import floor, log10

def get_prec(v, base):
    '''gives the precision value
    args:
       (v) as value needing precision in float
       (base) as the base value e.g. 0.05'''
    
    return round(round((v)/ base) * base, -int(floor(log10(base))))

In [1]:
# get the remaining quantities
from ib_insync import *
util.startLoop()

In [2]:
from helper import get_connected, get_snp_remqty, get_prec

In [3]:
ib = get_connected('snp', 'live')

name 'IB' is not defined
name 'IB' is not defined
name 'IB' is not defined
name 'IB' is not defined


UnboundLocalError: local variable 'ib' referenced before assignment

In [ ]:
with get_connected('snp', 'live') as ib:
    remqty = get_snp_remqty(ib)

In [ ]:
from os import listdir
import pandas as pd

from helper import grp_opts, get_prec

fspath = '../data/snp/' # path for snp pickles

minOptPrice = 0.1
minRom = 0.85

fs = listdir(fspath)

cols = ['optId', 'symbol', 'right', 'expiration', 'dte', 'strike', 'undPrice', 
'lo52', 'hi52', 'Fall', 'Rise', 'loFall', 'hiRise', 'std3', 'loStd3', 'hiStd3', 
'lotsize', 'optPrice', 'optMargin', 'rom']

optsList = [f for f in fs if f[-3:] == 'pkl']

df1 = pd.concat([pd.read_pickle(fspath+f) for f in optsList], axis=0, sort=True).reset_index(drop=True)[cols]

df2 = df1[((df1.strike > df1.hi52) | (df1.strike < df1.lo52)) & (df1.optPrice > minOptPrice) & (df1.rom > minRom)]

df2 = grp_opts(df2)

df2 = df2.assign(remqty=[remqty[u] for u in df2.symbol])

df3 = df2.groupby('symbol').apply(lambda x: x.nlargest(3, 'rom'))

df4 = df3.assign(expPrice=[get_prec(p*1.1, 0.05) for p in df3.optPrice])
df4 = df4.assign(expQty=(df4.remqty/3).astype('int'))

# high cost options
df_single = df4[df4.expQty == 0].set_index('optId').groupby('symbol').head(1)
df_single.loc[:, 'expQty'] = 1    # set quantity to 1

# remove high cost options
df_multiple = df4[df4.expQty != 0].set_index('optId')

# final set of options
df = pd.concat([df_single, df_multiple], axis=0).reset_index()

In [ ]:
df

In [ ]:
with IB().connect('127.0.0.1', 1300, clientId=2) as ib:
    contracts = [Contract(conId=i) for i in df.optId]
    qcs = ib.qualifyContracts(*contracts)
#     tickers = ib.reqTickers(*qcs)
    orders = [LimitOrder(action='SELL', totalQuantity=qty, lmtPrice=tgtPrice) for qty, tgtPrice in zip(df.expQty, df.expPrice)]